In [2]:
import pandas as pd
import numpy as np

In [3]:
df_21 = pd.read_csv('unduplicated data\data21nd.csv')
df_19 = pd.read_csv('unduplicated data\data19nd.csv')
# données non dupliquées

In [4]:
df_19 = df_19[~df_19['salary'].isnull()]
df_21 = df_21[~df_21['salary'].isnull()]
# on enlève les valeurs NaN des salaires

Notre objectif est d'entraîner un modèle de régression sur les données de la saison NBA 2018-2019, pour ensuite prédire les salaires de la saison 2020-2021 (nous n'utilisons pas la saison 2019-2020 qui a été perturbée par la crise du COVID). Pour ce faire, nous devons d'abord normaliser les salaires à cause de l'inflation.

In [5]:
df_19['salary']=(df_19['salary']-df_19['salary'].mean())/df_19['salary'].std()

# Machine Learning with Scikit Learn

### Creation of X (feature Matrix)

Nous avons sélectionné les quelques variables explicatives présentes dans nos données qui nous semblaient être les plus pertinentes pour juger la valeur salariale d'un joueur. De plus, nombre de variables sont très corrélées entre elles, ainsi les utiliser ensemble n'aurait que peu d'intérêt. Nous ne considérons également que les joueurs ayant comptabilisé plus de 20 matchs sur la saison en question.

In [6]:
x = df_19[df_19['games_played']>=20][['points_per_poss','total_rebounds_per_poss','assists_per_poss','points_per_game','reb_per_game','ast_per_game','win_shares','player_efficiency_rating']]

### Creation of Y (target)

In [7]:
y = df_19[df_19['games_played']>=20]['salary']

In [11]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

In [12]:
poss = ['points_per_poss','total_rebounds_per_poss','assists_per_poss']
game = ['points_per_game','reb_per_game','ast_per_game']

In [13]:
# pour les statistiques par possession
x_train1 = x_train.drop(game, axis=1)
x_test1 = x_test.drop(game, axis=1)

In [14]:
# pour les statistiques par match
x_train2 = x_train.drop(poss, axis=1)
x_test2 = x_test.drop(poss, axis=1)

## Machine Learning Regressors : premières variables

Nous essayons ici différents modèles de Machine Learning utilisant des méthodes de régression pour tenter de prédire les salaires des joueurs.

In [15]:
np.random.seed(53)

In [16]:
# 1
from sklearn import svm
reg1 = svm.SVR(gamma = 'scale')
reg1 = reg1.fit(x_train1, y_train)

In [17]:
y_preds = reg1.predict(x_test1)

In [18]:
reg1.score(x_test1,y_test)

0.27850413658724327

In [19]:
np.random.seed(37)

In [20]:
# 2
from sklearn.linear_model import LinearRegression
reg2 = LinearRegression()
reg2 = reg2.fit(x_train1,y_train)

In [21]:
y_preds2  = reg2.predict(x_test1)

In [22]:
reg2.score(x_test1, y_test)

0.32544652883902525

In [23]:
np.random.seed(42)

In [24]:
#3
from sklearn.ensemble import RandomForestRegressor
reg3 = RandomForestRegressor(n_estimators = 100)
reg3 = reg3.fit(x_train1, y_train)

In [25]:
y_preds3 = reg3.predict(x_test1)

In [26]:
reg3.score(x_test1, y_test)

0.312627739610403

Après ces premiers tests, nous nous sommes dit qu'il était très commun au basket de comparer les performances des joueurs en utilisant trois variables : les points par match, les passes par match et les rebonds par match. Nous avons alors créé ces trois nouvelles variables (modified data) qui n'étaient initialement pas dans notre base de données, et les avons utilisés pour nos modèles.

### Nouvelles variables

In [27]:
np.random.seed(53)

In [28]:
# 1
from sklearn import svm
reg4 = svm.SVR(gamma = 'scale')
reg4 = reg4.fit(x_train2, y_train)

In [29]:
y_preds4 = reg4.predict(x_test2)

In [30]:
reg4.score(x_test2,y_test)

0.31456995130108945

In [31]:
np.random.seed(37)

In [32]:
# 2
from sklearn.linear_model import LinearRegression
reg5 = LinearRegression()
reg5 = reg5.fit(x_train2,y_train)

In [33]:
y_preds5  = reg5.predict(x_test2)

In [34]:
reg5.score(x_test2, y_test)

0.3463385961314126

In [35]:
np.random.seed(42)

In [36]:
# 3
from sklearn.ensemble import RandomForestRegressor
reg6 = RandomForestRegressor(n_estimators = 100)
reg6 = reg6.fit(x_train2, y_train)

In [37]:
y_preds6 = reg6.predict(x_test2)

In [38]:
reg6.score(x_test2, y_test)

0.21936101405410413

Après avoir essayer ces trois modèles, le plus efficace (lorsque l'on se base sur le R-squared de la régression) semble être le sklearn.linear_model.LinearRegression.

**Tableau récapitulatif des score des modèles :**

In [52]:
res = pd.DataFrame(columns=['svm','RegLin','RandomForest'],index=['poss','game'])
res['svm'] = [reg1.score(x_test1,y_test),reg4.score(x_test2,y_test)]
res['RegLin'] = [reg2.score(x_test1,y_test),reg5.score(x_test2,y_test)]
res['RandomForest'] = [reg3.score(x_test1,y_test),reg6.score(x_test2,y_test)]

In [55]:
res.to_csv(r'model_results.csv')

## Improving our model

In [39]:
reg5.get_params()

{'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'normalize': False}

In [45]:
from sklearn.linear_model import LinearRegression
reg7 = LinearRegression(normalize = True)
reg7 = reg7.fit(x_train2,y_train)

In [46]:
reg7.score(x_test2,y_test)

0.3463385961314125

In [47]:
from sklearn.linear_model import LinearRegression
reg8 = LinearRegression(fit_intercept = False)
reg8 = reg8.fit(x_train2,y_train)

In [48]:
reg7.score(x_test2,y_test)

0.3463385961314125

Visiblement, changer les paramètres par défaut du modèle ne modifie pas significativement sa performance.

In [49]:
# On enregistre le modèle entraîné pour le réutiliser dans le projet final

In [56]:
import pickle

In [57]:
pickle.dump(reg5, open('reg_linear_regression.pkl', 'wb'))